# Часть 2. NLP.

Используем набор данных из первой части задания. 

Построим линейную регрессию с регуляцизацией, которая будет угадывать P/E по ключевым словам текста описаний факторов роста/падения компаний.

С помощью TF-IDF построим Document-Term-Matrix, ограниченную 200 словами + биграммами.

In [ ]:
import os
import re
from functools import reduce
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

NUM_WORKERS = os.cpu_count()

## Необходимые определения:

In [2]:
regex_for_words_only = re.compile(r'[А-Яа-яA-zёЁ]+[А-Яа-яA-zёЁ-]*[А-Яа-яA-zёЁ]+') # ёЁ идут отдельно!

def words_only(text, regex=regex_for_words_only):
    try:
        return " ".join(regex.findall(text)).lower() # to lower case
    except:
        return ""
    

def process_data(data, word_tokenizer, stop_words, stemmer):
    result = []

    for item in tqdm(data):
        text_lower = words_only(item) # оставим только слова
        tokens     = word_tokenizer.tokenize(text_lower) #разбиваем текст на слова

        # удаляем пунктуацию и стоп-слова
        tokens = [word for word in tokens if (word not in stop_words and not word.isnumeric())]

        # удаляем односимвольные токены

        tokens = list(filter(lambda s: len(s) > 1, tokens))

        # выполняем word-stem
        tokens = list(map(lambda s: stemmer.stem(s), tokens))
    
        result.append(' '.join(tokens))

    return result


def predict(lin_reg, X):
    y_pred = lin_reg.predict(X)
    return pd.Series(y_pred).astype(np.float32)

## Предобработка данных

Токенизируем по словам, применяем стоп-лист (скачан из Интернет и дополнен), приводим всё к нижнему регистру, удаляем всё, кроме букв, удаляем односимвольные слова, применяем word-stem.

In [3]:
df = pd.read_csv('pe_by_ticket.csv')
df = df.drop(columns=['Ticket'])

with open('stopWords.txt', 'rt') as file_stop_words:
    stop_words = list(map(lambda line: line[:-1], file_stop_words.readlines())) # Удалили \n в конце строк

print('Stop list: ...', stop_words[40:50], '...')

# инициализируем tokenizer
word_tokenizer = nltk.WordPunctTokenizer()

# инициализируем стеммер
stemmer = SnowballStemmer("russian")

texts = process_data(df['Description'], word_tokenizer, stop_words, stemmer)
y = df['P/E']

print('texts[0] =', texts[0])
print('y[0] =', y[0])



Stop list: ... ['млн', 'оао', 'пао', 'а', 'абсолютно', 'авторизоваться', 'активный', 'алло', 'алтухов', 'атмосфера'] ...


100%|██████████| 100/100 [00:00<00:00, 289.83it/s]

texts[0] = фактор рост паден акц перешел выплат дивиденд приб начин вышел прибыл октябр выплат дивиденд рекордн прибыл ожида рекордн дивиденд плат чист приб высок roe высок достаточн капита замедлен кредитован стран снижа рост кредитн портфел процентн доход сбер рост процентн ставок чист процентн марж прибыл сбер след ипотек основ розничн кредитн портфел средн срок ипотечн кредит средн выросл последн вырос риск нача проблем выплат
y[0] = 3.52


## Разбиваем набор данных на train, test и переводим текст в матрицы через TF-IDF

In [10]:
X_train_texts, X_test_texts, y_train, y_test = train_test_split(texts, y, test_size=0.2, random_state=42)

#print(texts[:10])
#print(X_test_texts[:10])

# Fit TF-IDF on train texts
vectorizer = TfidfVectorizer(max_features = 200, 
                             ngram_range=(1, 2), 
                             norm = None, # we need disable it, because default value is ’l2’
                             lowercase=False,
                             dtype=np.float32) # возмем топ 200 слов + биграмы
vectorizer.fit(X_train_texts)

# Топ-10 слов
print('Top 10:', vectorizer.get_feature_names_out()[:10])

# Обучаем TF-IDF на train, а затем применяем к train и test
X_train_texts = vectorizer.transform(X_train_texts)

X_test_texts = vectorizer.transform(X_test_texts)

# Пример
print('First object:\n', X_train_texts[0].todense()[:1]) # посмотрим на первую строку
print(X_train_texts[0].shape)

Top 10: ['capex' 'ebitda' 'float' 'free' 'free float' 'ipo' 'автомобил' 'актив'
 'акц' 'акционер']
First object:
 [[ 0.         0.         0.         0.         0.         0.
   0.         0.         1.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         9.03582    4.1972246  0.         0.         0.
   0.         0.         0.         0.         3.448539   0.
   0.         0.         0.         0.         3.7850113  0.
   0.         0.         0.         0.         2.7553918  0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         3.7850113  3.7850113  0.
   0.         0.         7.5700226  0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         2.0986123  0.         0.         0.         0.
   0.         0.         0.     

## Создадим линейную регрессию с регуляризацией Lasso (L1)

In [33]:
lin_reg = Lasso(alpha=1, random_state=1)
lin_reg.fit(X_train_texts, y_train)

Lasso(alpha=1, random_state=1)

In [37]:
y_pred_train = predict(lin_reg, X_train_texts)
print('RMSE score на тренировочной выборке\t: {:.3f}'.format(root_mean_squared_error(y_train, y_pred_train)))
print('R2 score на тренировочной выборке\t: {:.3f}'.format(r2_score(y_train, y_pred_train)))

RMSE score на тренировочной выборке	: 9.320
R2 score на тренировочной выборке	: 0.922


## Проверим модель на отложенной тестовой выборке

In [34]:
y_pred_test = predict(lin_reg, X_test_texts)
print('RMSE score на тестовой выборке\t: {:.3f}'.format(root_mean_squared_error(y_test, y_pred_test)))
print('R2 score на тестовой выборке\t: {:.3f}'.format(r2_score(y_test, y_pred_test)))

RMSE score на тестовой выборке	: 17.474
R2 score на тестовой выборке	: -0.799


In [36]:
first_n = 5
for pred, test in zip(y_pred_test[:first_n], y_test[:first_n]):
    print(pred, test)

13.224664688110352 22.8
8.144805908203125 4.44
8.050402641296387 5.3
4.723942756652832 4.67
6.842401504516602 2.68
